# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [5]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [52]:
publications = pd.read_csv("Papers_Publications_Date_Dash_Format.csv", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url,category
0,2024-01-30,BeGrading: Large Language Models for Enhanced ...,Neural Computing and Applications,This paper explores the application of LLMs to...,"Mina Yousef, Kareem Mohamed, Walaa Medhat, Ens...",begrading-llm-feedback,https://doi.org/10.1007/s00521-024-10449-y,-,manuscripts
1,2024-02-15,Hands-on Analysis of Using Large Language Mode...,Information Systems,This paper investigates the use of LLMs for au...,"Kareem Mohamed, Mina Yousef, Walaa Medhat, Ens...",auto-evaluation-llm,https://doi.org/10.1016/j.is.2024.102473,-,manuscripts
2,2024-03-10,SliceInsightNet: Advanced Hybrid Image Segment...,2024 IEEE International Conference on Future M...,This paper presents a hybrid image segmentatio...,"Mina Yousef & Essam A. Rashed. (2024). ""SliceI...",sliceinsightnet-segmentation,https://ieeexplore.ieee.org/abstract/document/...,-,conferences
3,2024-03-20,Optimizing Real-Time Data Processing in Resour...,2024 IEEE International Conference on Future M...,This paper explores a Spark and GPU-driven wor...,"Mina Yousef, Ghada Khoriba, & Tamer Arafa. (20...",real-time-data-processing,https://ieeexplore.ieee.org/abstract/document/...,-,conferences
4,2022-06-15,Deep Learning Approaches for Analyzing Visual ...,"Mobile, Intelligent, and Ubiquitous Computing ...",This paper discusses the use of deep learning ...,"Andrew Karam, Kirollos Boles, Mario Raouf, Min...",deep-learning-visual-stimuli,-,-,conferences
5,2020-11-10,Natural Language Processing: Text Categorizati...,International Journal of Advanced Networking a...,This paper covers advanced techniques for text...,"Mona Nasr, Andrew Karam, Mina Atef, Kirollos B...",nlp-text-categorization,-,-,manuscripts
6,2023-05-10,Generating Application Traffic Data from Tax D...,"NetMob 2023 Data Challenge, Madrid, Spain",This project introduces a synthetic data appro...,"Tamer Arafa, Noha Gamal, Ghada Khoriba, Mina A...",synthetic-data-approach,-,-,challenge
7,2023-05-15,Désordre: A Framework for Real-Time Detection ...,"NetMob 2023 Data Challenge, Madrid, Spain",This framework focuses on real-time detection ...,"Noha Gamal, Mina Atef Yousef, & Ahmed El-Mahdy...",real-time-detection-social-unrest,-,-,challenge
8,2023-05-20,Predicting the Productivity Indicators of a So...,"NetMob 2023 Data Challenge, Madrid, Spain",This project predicts productivity indicators ...,"Noha Gamal, Tamer Arafa, Mina Yousef, & Ahmed ...",productivity-indicators-dynamics,-,-,challenge


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [53]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [54]:
publications

,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url,category
0,2024-01-30,BeGrading: Large Language Models for Enhanced ...,Neural Computing and Applications,This paper explores the application of LLMs to...,"Mina Yousef, Kareem Mohamed, Walaa Medhat, Ens...",begrading-llm-feedback,https://doi.org/10.1007/s00521-024-10449-y,-,manuscripts
1,2024-02-15,Hands-on Analysis of Using Large Language Mode...,Information Systems,This paper investigates the use of LLMs for au...,"Kareem Mohamed, Mina Yousef, Walaa Medhat, Ens...",auto-evaluation-llm,https://doi.org/10.1016/j.is.2024.102473,-,manuscripts
2,2024-03-10,SliceInsightNet: Advanced Hybrid Image Segment...,2024 IEEE International Conference on Future M...,This paper presents a hybrid image segmentatio...,"Mina Yousef & Essam A. Rashed. (2024). ""SliceI...",sliceinsightnet-segmentation,https://ieeexplore.ieee.org/abstract/document/...,-,conferences
3,2024-03-20,Optimizing Real-Time Data Processing in Resour...,2024 IEEE International Conference on Future M...,This paper explores a Spark and GPU-driven wor...,"Mina Yousef, Ghada Khoriba, & Tamer Arafa. (20...",real-time-data-processing,https://ieeexplore.ieee.org/abstract/document/...,-,conferences
4,2022-06-15,Deep Learning Approaches for Analyzing Visual ...,"Mobile, Intelligent, and Ubiquitous Computing ...",This paper discusses the use of deep learning ...,"Andrew Karam, Kirollos Boles, Mario Raouf, Min...",deep-learning-visual-stimuli,-,-,conferences
5,2020-11-10,Natural Language Processing: Text Categorizati...,International Journal of Advanced Networking a...,This paper covers advanced techniques for text...,"Mona Nasr, Andrew Karam, Mina Atef, Kirollos B...",nlp-text-categorization,-,-,manuscripts
6,2023-05-10,Generating Application Traffic Data from Tax D...,"NetMob 2023 Data Challenge, Madrid, Spain",This project introduces a synthetic data appro...,"Tamer Arafa, Noha Gamal, Ghada Khoriba, Mina A...",synthetic-data-approach,-,-,challenge
7,2023-05-15,Désordre: A Framework for Real-Time Detection ...,"NetMob 2023 Data Challenge, Madrid, Spain",This framework focuses on real-time detection ...,"Noha Gamal, Mina Atef Yousef, & Ahmed El-Mahdy...",real-time-detection-social-unrest,-,-,challenge
8,2023-05-20,Predicting the Productivity Indicators of a So...,"NetMob 2023 Data Challenge, Madrid, Spain",This project predicts productivity indicators ...,"Noha Gamal, Tamer Arafa, Mina Yousef, & Ahmed ...",productivity-indicators-dynamics,-,-,challenge


In [55]:
import os
for row, item in publications.iterrows():
    print(item)
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications\n"""
    md += """category: """+html_escape(item.category)
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

pub_date                                             2024-01-30
title         BeGrading: Large Language Models for Enhanced ...
venue                         Neural Computing and Applications
excerpt       This paper explores the application of LLMs to...
citation      Mina Yousef, Kareem Mohamed, Walaa Medhat, Ens...
url_slug                                 begrading-llm-feedback
paper_url            https://doi.org/10.1007/s00521-024-10449-y
slides_url                                                    -
category                                            manuscripts
Name: 0, dtype: object
pub_date                                             2024-02-15
title         Hands-on Analysis of Using Large Language Mode...
venue                                       Information Systems
excerpt       This paper investigates the use of LLMs for au...
citation      Kareem Mohamed, Mina Yousef, Walaa Medhat, Ens...
url_slug                                    auto-evaluation-llm
paper_url        

These files are in the publications directory, one directory below where we're working from.

In [16]:
!ls ../_publications/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
